In [15]:
import nbimporter

In [16]:
import utils
import blockchain

### PARSE RESOURCES

In [17]:
binary_interfaces = utils.load_json('resources/binary.json')

In [18]:
settings = utils.load_json('resources/settings.json')

### CONNECT WITH A BLOCKCHAIN GATEWAY

In [19]:
web3_instance = blockchain.connect(settings)

CONNECTION ESTABLISHED!


### SERIALIZE THE DEVICE MANAGER

In [20]:
device_manager = blockchain.contract(
    binary_interfaces['devicemanager'],
    web3_instance, settings
)

### CREATE A NEW DEVICE CONTRACT

In [26]:
identifier = 'another-identifier'

In [27]:
name = 'foobar'

In [28]:
device_manager.write({
    'func': 'create_device',
    'args': [identifier, name]
})

web3.exceptions.ContractLogicError('execution reverted: VM Exception while processing transaction: revert this identifier already exists')

### FETCH YOUR DEVICE COLLECTION (IDENTIFIERS)

In [29]:
collection =  device_manager.read({
    'func': 'fetch_collection',
    'args': settings['keys']['public']
})

In [30]:
collection

['another-identifier']

### FETCH THE CREATED DEVICE CONTRACT'S ADDRESS BY USING THE IDENTIFIER

In [33]:
device_address =  device_manager.read({
    'func': 'fetch_device',
    'args': identifier
})

In [34]:
device_address

'0xCa24282be001a428d27FdAC4676561c2EB1DE393'

### SERIALIZE THE DEVICE CONTRACT

In [35]:
device_contract = blockchain.contract({
    'address': device_address,
    'abi': binary_interfaces['device']['abi']
}, web3_instance, settings)

### FETCH THE DEVICE CONTRACT'S DETAILS

In [37]:
details = device_contract.read('details')

In [38]:
print('OWNER:\t\t', details[0])
print('IDENTIFIER:\t', details[1])
print('NAME:\t\t', details[2])
print('CREATED:\t', details[3])
print('ACTIVE:\t\t', details[4])

OWNER:		 0x1dF62f291b2E969fB0849d99D9Ce41e2F137006e
IDENTIFIER:	 another-identifier
NAME:		 foobar
CREATED:	 10
ACTIVE:		 False


### CHANGE THE DEVICE CONTRACT ACTIVE STATUS

In [39]:
device_contract.write('toggle_active')

AttributeDict({'transactionHash': HexBytes('0x2fd0bd0103cd68ce333ad5b5e8104442c392d139dfc24319abbd17ab8a75b1a6'),
 'transactionIndex': 0,
 'blockHash': HexBytes('0xb64dbf0cb436d77671847716bf25a8dbef576d24e4719a4f530a8d3f1440a13d'),
 'blockNumber': 11,
 'from': '0x1dF62f291b2E969fB0849d99D9Ce41e2F137006e',
 'to': '0xCa24282be001a428d27FdAC4676561c2EB1DE393',
 'gasUsed': 42887,
 'cumulativeGasUsed': 42887,
 'contractAddress': None,
 'logs': [],
 'status': 1,
 'logsBloom': HexBytes('0x00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000'

### CHECK ACTIVE STATUS AGAIN

In [40]:
status = device_contract.read('active')

In [41]:
status

True